In [1]:
import numpy as np

In [2]:
import ipyvolume as ipv

In [3]:
import matplotlib.pyplot as plt

In [4]:
%matplotlib widget

In [9]:
import xarray as xr
import h5py

In [23]:
def hdf5_to_xarray(h5_data):
    """Convert hdf5 dataset to xarray dataset."""
    data = h5_data[:]
    coords = [(dim.label, dim[0][:]) for dim in h5_data.dims]
    xr_data = xr.DataArray(data, coords=[coords[0], coords[3], coords[2], coords[1]])
    return xr_data

In [27]:
def get_data(setnum, task, skip=1):
    # Load file
    filename = "scripts/rbc/rbc_s%i.h5" %setnum
    file = h5py.File(filename, 'r')
    # Get data
    #return file['tasks'][task]
    xr = hdf5_to_xarray(file['tasks'][task])
    return xr

In [54]:
T = get_data(1, 'T')
T.coords['z'] = T.coords['r'] * np.cos(T.coords['theta'])
T.coords['x'] = T.coords['r'] * np.sin(T.coords['theta']) * np.cos(T.coords['phi'])

In [58]:
plt.figure()
for i in range(10, 21):
    plt.clf()
    (T.isel(phi=0, t=i) - T.coords['z']).plot(x='x', y='z')
    (T.isel(phi=92, t=i) - T.coords['z']).plot(x='x', y='z')
    plt.axis('equal')
    plt.savefig('frame_%i.png' %i)

/Users/kburns/Software/miniconda3/envs/dedalus-openblas/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

###### 

In [8]:
def volbot(setnum, task, skip=1):
    data = get_data(setnum, task, skip=skip)
    # Correct z offset
    data = np.roll(data, 64//skip, axis=2)
    # Renormalize
    data /= np.sqrt(np.mean(data**2))
    data[data < -2] = -2
    data[data > 2] = 2
    # Volume rendering
    ipv.figure(width=1000, height=1000)
    ipv.style.axes_off()
    vol = ipv.volshow(data, level=[0.25, 0.25, 0.75], opacity=[0.1, 0.02, 0.1], level_width=0.05)
    vol.brightness = 2.0
    ipv.pylab.view(azimuth=90, elevation=30, distance=2)
    ipv.show()

In [ ]:
volbot(7, 'ωz', skip=4)